# 🔧 WSPlumber - Backtesting con GPU

Este notebook permite ejecutar backtesting acelerado del sistema WSPlumber usando:
- **Polars** para procesamiento de datos (más rápido que Pandas)
- **GPU** para cálculos paralelos
- **Parquet** como formato de datos históricos

---

## 1. Setup Inicial

In [ ]:
# Verificar GPU disponible
!nvidia-smi

In [ ]:
# Instalar dependencias
!pip install polars pyarrow cudf-cu12 --quiet
!pip install numba --quiet

In [ ]:
# Clonar repositorio (privado - necesita token)
# Opción 1: Repo público
# !git clone https://github.com/tu-usuario/wsplumber.git

# Opción 2: Repo privado con token
# from google.colab import userdata
# GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
# !git clone https://{GITHUB_TOKEN}@github.com/tu-usuario/wsplumber.git

# Opción 3: Subir archivos manualmente
from google.colab import files
print("Sube tu archivo Parquet de tick data:")
# uploaded = files.upload()

## 2. Cargar Datos Históricos

In [ ]:
import polars as pl
from pathlib import Path

# Cargar Parquet
# PARQUET_PATH = "wsplumber/data/EURUSD_ticks.parquet"
PARQUET_PATH = "/content/EURUSD_ticks.parquet"  # Si subiste manualmente

df = pl.read_parquet(PARQUET_PATH)
print(f"Ticks cargados: {len(df):,}")
print(f"Rango: {df['timestamp'].min()} - {df['timestamp'].max()}")
df.head()

In [ ]:
# Estadísticas del dataset
print("Estadísticas:")
print(f"  Ticks totales: {len(df):,}")
print(f"  Memoria: {df.estimated_size() / 1024 / 1024:.2f} MB")
print(f"  Columnas: {df.columns}")

## 3. Estrategia de Backtesting (GPU Accelerated)

In [ ]:
from numba import cuda, jit
import numpy as np

# Parámetros de estrategia
TP_PIPS = 10
SL_PIPS = 50
PIP_SIZE = 0.0001  # Para pares no-JPY

@jit(nopython=True)
def simulate_trades_cpu(bids: np.ndarray, asks: np.ndarray) -> tuple:
    """
    Simula trades: abre en cada tick, cierra cuando alcanza TP o SL.
    Versión CPU con Numba JIT.
    """
    n = len(bids)
    trades = []
    total_pnl = 0.0
    wins = 0
    losses = 0
    
    i = 0
    while i < n - 1:
        # Abrir BUY al ask
        entry_price = asks[i]
        tp_price = entry_price + (TP_PIPS * PIP_SIZE)
        sl_price = entry_price - (SL_PIPS * PIP_SIZE)
        
        # Buscar cierre
        for j in range(i + 1, n):
            if bids[j] >= tp_price:  # TP alcanzado
                total_pnl += TP_PIPS
                wins += 1
                i = j + 1
                break
            elif bids[j] <= sl_price:  # SL alcanzado
                total_pnl -= SL_PIPS
                losses += 1
                i = j + 1
                break
        else:
            # Sin cierre, saltar al final
            break
    
    return total_pnl, wins, losses

print("Función JIT compilada ✓")

In [ ]:
# Ejecutar simulación
bids = df['bid'].to_numpy()
asks = df['ask'].to_numpy()

import time
start = time.time()
pnl, wins, losses = simulate_trades_cpu(bids, asks)
elapsed = time.time() - start

print(f"\n📊 RESULTADOS DEL BACKTEST")
print(f"="*40)
print(f"  Tiempo de ejecución: {elapsed:.3f}s")
print(f"  Ticks procesados: {len(bids):,}")
print(f"  Velocidad: {len(bids)/elapsed:,.0f} ticks/segundo")
print(f"")
print(f"  🏆 Total trades: {wins + losses}")
print(f"  ✅ Wins: {wins} ({100*wins/(wins+losses):.1f}%)")
print(f"  ❌ Losses: {losses} ({100*losses/(wins+losses):.1f}%)")
print(f"  💰 P&L Total: {pnl:.1f} pips")

## 4. Versión GPU (CUDA) - Para datasets grandes

In [ ]:
# Verificar si cuDF está disponible (GPU DataFrame)
try:
    import cudf
    print("cuDF disponible ✓ - Usando GPU")
    
    # Cargar datos en GPU
    df_gpu = cudf.read_parquet(PARQUET_PATH)
    print(f"Datos cargados en GPU: {len(df_gpu):,} ticks")
    
except ImportError:
    print("cuDF no disponible - Usando Polars (CPU)")
    print("Para usar GPU, asegúrate de tener runtime GPU en Colab")

In [ ]:
@cuda.jit
def check_tp_sl_kernel(bids, asks, tp_pips, sl_pips, pip_size, results):
    """
    Kernel CUDA para verificar TP/SL en paralelo.
    Cada hilo procesa un rango de ticks.
    """
    idx = cuda.grid(1)
    if idx < len(bids):
        entry = asks[idx]
        tp = entry + (tp_pips * pip_size)
        sl = entry - (sl_pips * pip_size)
        
        # Buscar en los siguientes 1000 ticks
        for j in range(idx + 1, min(idx + 1000, len(bids))):
            if bids[j] >= tp:
                results[idx] = 1  # Win
                return
            elif bids[j] <= sl:
                results[idx] = -1  # Loss
                return
        
        results[idx] = 0  # No cierre

## 5. Análisis de Resultados

In [ ]:
import matplotlib.pyplot as plt

# Gráfico de equity curve (simplificado)
def plot_equity_curve(pnl_history: list):
    cumulative = np.cumsum(pnl_history)
    
    plt.figure(figsize=(12, 4))
    plt.plot(cumulative, color='green' if cumulative[-1] > 0 else 'red')
    plt.title('Curva de Equity (Pips Acumulados)')
    plt.xlabel('Trade #')
    plt.ylabel('Pips')
    plt.axhline(y=0, color='gray', linestyle='--')
    plt.grid(True, alpha=0.3)
    plt.show()

# Ejemplo con datos simulados
sample_pnl = [10, 10, -50, 10, 10, 10, -50, 10, 10, 10]
plot_equity_curve(sample_pnl)

## 6. Optimización de Parámetros (Grid Search)

In [ ]:
# Grid search de TP/SL
results = []

for tp in [5, 10, 15, 20]:
    for sl in [30, 50, 80, 100]:
        # Modificar parámetros globales (simplificado)
        TP_PIPS = tp
        SL_PIPS = sl
        
        pnl, wins, losses = simulate_trades_cpu(bids[:100000], asks[:100000])
        
        results.append({
            'tp': tp,
            'sl': sl,
            'pnl': pnl,
            'win_rate': wins / (wins + losses) if (wins + losses) > 0 else 0,
            'trades': wins + losses
        })

results_df = pl.DataFrame(results)
print("Grid Search Results:")
print(results_df.sort('pnl', descending=True))

---

## 📌 Notas

- Para usar **GPU**, selecciona `Runtime > Change runtime type > GPU` en Colab
- Los archivos Parquet deben subirse o conectarse desde Google Drive
- La versión CUDA es más eficiente con datasets > 1M de ticks